In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
import numpy as np 
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt 
import seaborn as sns 
import yfinance as yf 
import yahoo_fin.stock_info as si 
from bs4 import BeautifulSoup 
from yahoo_fin.stock_info import get_analysts_info
import requests
import json

In [2]:
url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol=AAPL&apikey=N9VFU80S37LZPISJ'
r = requests.get(url)
data = r.json()

print(data)

{'symbol': 'AAPL', 'annualEarnings': [{'fiscalDateEnding': '2023-12-31', 'reportedEPS': '0'}, {'fiscalDateEnding': '2023-09-30', 'reportedEPS': '6.12'}, {'fiscalDateEnding': '2022-09-30', 'reportedEPS': '6.11'}, {'fiscalDateEnding': '2021-09-30', 'reportedEPS': '5.62'}, {'fiscalDateEnding': '2020-09-30', 'reportedEPS': '3.27'}, {'fiscalDateEnding': '2019-09-30', 'reportedEPS': '2.98'}, {'fiscalDateEnding': '2018-09-30', 'reportedEPS': '2.97'}, {'fiscalDateEnding': '2017-09-30', 'reportedEPS': '2.3'}, {'fiscalDateEnding': '2016-09-30', 'reportedEPS': '2.0675'}, {'fiscalDateEnding': '2015-09-30', 'reportedEPS': '2.3'}, {'fiscalDateEnding': '2014-09-30', 'reportedEPS': '1.6075'}, {'fiscalDateEnding': '2013-09-30', 'reportedEPS': '1.415'}, {'fiscalDateEnding': '2012-09-30', 'reportedEPS': '1.5775'}, {'fiscalDateEnding': '2011-09-30', 'reportedEPS': '0.9875'}, {'fiscalDateEnding': '2010-09-30', 'reportedEPS': '0.54'}, {'fiscalDateEnding': '2009-09-30', 'reportedEPS': '0.2225'}, {'fiscalDate

In [3]:
data.pop("annualEarnings")

[{'fiscalDateEnding': '2023-12-31', 'reportedEPS': '0'},
 {'fiscalDateEnding': '2023-09-30', 'reportedEPS': '6.12'},
 {'fiscalDateEnding': '2022-09-30', 'reportedEPS': '6.11'},
 {'fiscalDateEnding': '2021-09-30', 'reportedEPS': '5.62'},
 {'fiscalDateEnding': '2020-09-30', 'reportedEPS': '3.27'},
 {'fiscalDateEnding': '2019-09-30', 'reportedEPS': '2.98'},
 {'fiscalDateEnding': '2018-09-30', 'reportedEPS': '2.97'},
 {'fiscalDateEnding': '2017-09-30', 'reportedEPS': '2.3'},
 {'fiscalDateEnding': '2016-09-30', 'reportedEPS': '2.0675'},
 {'fiscalDateEnding': '2015-09-30', 'reportedEPS': '2.3'},
 {'fiscalDateEnding': '2014-09-30', 'reportedEPS': '1.6075'},
 {'fiscalDateEnding': '2013-09-30', 'reportedEPS': '1.415'},
 {'fiscalDateEnding': '2012-09-30', 'reportedEPS': '1.5775'},
 {'fiscalDateEnding': '2011-09-30', 'reportedEPS': '0.9875'},
 {'fiscalDateEnding': '2010-09-30', 'reportedEPS': '0.54'},
 {'fiscalDateEnding': '2009-09-30', 'reportedEPS': '0.2225'},
 {'fiscalDateEnding': '2008-09-30'

In [6]:
data

{'symbol': 'AAPL',
 'quarterlyEarnings': [{'fiscalDateEnding': '2023-09-30',
   'reportedDate': '2023-11-02',
   'reportedEPS': '1.46',
   'estimatedEPS': '1.39',
   'surprise': '0.07',
   'surprisePercentage': '5.036'},
  {'fiscalDateEnding': '2023-06-30',
   'reportedDate': '2023-08-03',
   'reportedEPS': '1.26',
   'estimatedEPS': '1.19',
   'surprise': '0.07',
   'surprisePercentage': '5.8824'},
  {'fiscalDateEnding': '2023-03-31',
   'reportedDate': '2023-05-04',
   'reportedEPS': '1.52',
   'estimatedEPS': '1.43',
   'surprise': '0.09',
   'surprisePercentage': '6.2937'},
  {'fiscalDateEnding': '2022-12-31',
   'reportedDate': '2023-02-02',
   'reportedEPS': '1.88',
   'estimatedEPS': '1.94',
   'surprise': '-0.06',
   'surprisePercentage': '-3.0928'},
  {'fiscalDateEnding': '2022-09-30',
   'reportedDate': '2022-10-27',
   'reportedEPS': '1.29',
   'estimatedEPS': '1.27',
   'surprise': '0.02',
   'surprisePercentage': '1.5748'},
  {'fiscalDateEnding': '2022-06-30',
   'reported

In [13]:
df = pd.DataFrame.from_dict(data, orient='columns')

In [14]:
df

,symbol,quarterlyEarnings
0,AAPL,"{'fiscalDateEnding': '2023-09-30', 'reportedDa..."
1,AAPL,"{'fiscalDateEnding': '2023-06-30', 'reportedDa..."
2,AAPL,"{'fiscalDateEnding': '2023-03-31', 'reportedDa..."
3,AAPL,"{'fiscalDateEnding': '2022-12-31', 'reportedDa..."
4,AAPL,"{'fiscalDateEnding': '2022-09-30', 'reportedDa..."
...,...,...
106,AAPL,"{'fiscalDateEnding': '1997-03-31', 'reportedDa..."
107,AAPL,"{'fiscalDateEnding': '1996-12-31', 'reportedDa..."
108,AAPL,"{'fiscalDateEnding': '1996-09-30', 'reportedDa..."
109,AAPL,"{'fiscalDateEnding': '1996-06-30', 'reportedDa..."
